In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy
import scipy.stats as stats
from scipy.stats import linregress
from scipy.stats import ttest_ind
import sys
import os
import re

In [2]:
#Note: For the entirety of this notebook, the words "collab" and "collaboration" 
    #refer to songs with three or more credited artists. "Non-solo" songs are any
    #songs with more than one contributor- both duets and collabs are "non-solo" 
    #songs. -Kelly

In [3]:
#read in data.csv
csv_path = "Resources/data.csv"
data_df = pd.read_csv(csv_path)

data_df

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.98200,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.6650,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.73200,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.1600,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.96100,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.1010,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.96700,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.3810,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.95700,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.2290,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170648,0.6080,2020,0.08460,"['Anuel AA', 'Daddy Yankee', 'KAROL G', 'Ozuna...",0.786,301714,0.808,0,0KkIkfsLEJbrcIhYsCL7L5,0.000289,7,0.0822,-3.702,1,China,72,2020-05-29,0.0881,105.029
170649,0.7340,2020,0.20600,['Ashnikko'],0.717,150654,0.753,0,0OStKKAuXlxA0fMH54Qs6E,0.000000,7,0.1010,-6.020,1,Halloweenie III: Seven Days,68,2020-10-23,0.0605,137.936
170650,0.6370,2020,0.10100,['MAMAMOO'],0.634,211280,0.858,0,4BZXVFYCb76Q0Klojq4piV,0.000009,4,0.2580,-2.226,0,AYA,76,2020-11-03,0.0809,91.688
170651,0.1950,2020,0.00998,['Eminem'],0.671,337147,0.623,1,5SiZJoLXp3WOl3J4C8IK0d,0.000008,2,0.6430,-7.161,1,Darkness,70,2020-01-17,0.3080,75.055


In [4]:
# #Only run this code if the newdata_df hasn't been created yet. -Kelly

# #create new row with the number of artists performing on each track

# #create a new dataframe with only the "artists" column from data.csv to make it easier to work with
# artist_df = pd.DataFrame(data_df['artists'])
# artist_df["collaborators"] = ""

# #count number of artists per track by counting the number of commas in the "artists" column for each row, then adding 1.
# for index, row in artist_df.iterrows():
#     row["collaborators"] = row["artists"].count(",") + 1

# #join the two dataframes together and drop the duplicate "artists" column
# new_data_df = data_df.join(artist_df, lsuffix="DROP").filter(regex="^(?!.*DROP)")

# new_data_df.to_csv(r'Resources\data_with_collab.csv')
csv_path = "Resources/data_with_collab.csv"

In [5]:
#read in new csv and remove extra index column
newdata_df = pd.read_csv(csv_path)
try:
    newdata_df = newdata_df.drop(columns=["Unnamed: 0"])
except:
    pass
newdata_df
# remove all single quotes from artist column so dataframes can be merged later
newdata_df['artists'] = newdata_df['artists'].str.strip("[]").str.replace("'","").str.replace('"',"")
newdata_df.sort_values('artists')

,valence,year,acousticness,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,artists,collaborators
38485,0.488,2020,0.810,0.847,120000,0.448,1,0j8ppsOOawdPCJnSTcXgOy,0.000013,0,0.3250,-10.140,1,GOSHA,78,2020-03-06,0.4160,79.975,$NOT,1
155042,0.698,2018,0.874,0.716,119683,0.414,1,4LS6xLmYwTUURADEIrmNaT,0.000042,10,0.1360,-12.536,1,VINTAGE DIOR,59,2018-08-20,0.3400,147.974,$NOT,1
140326,0.500,2018,0.814,0.819,120000,0.462,1,3q8ms2oRMFsyxUYYKDiXIP,0.000016,0,0.3180,-9.638,0,GOSHA,61,2018-09-14,0.4850,159.873,$NOT,1
92341,0.351,2020,0.290,0.829,148093,0.619,1,4X5AMbgW6whnAuDX10fhgq,0.000371,1,0.0975,-7.258,1,Revenge,70,2020-09-24,0.1980,96.035,$NOT,1
170607,0.631,2020,0.169,0.844,125853,0.720,1,3PXi72ZtSqx1PZc40KS0Qj,0.000000,1,0.0797,-7.418,0,Mean,65,2020-10-30,0.3000,99.917,"$NOT, Flo Milli",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144503,0.958,1963,0.720,0.816,147373,0.187,0,7jSt9XJHRynGrrcnnIpOcf,0.000000,9,0.1150,-20.530,0,丟丟銅仔,23,1963-05-28,0.0930,124.617,黃國隆,1
144488,0.704,1963,0.771,0.610,208760,0.175,0,6Pqs2suXEqCGx7Lxg5dlrB,0.000000,7,0.0309,-20.232,1,思想起,23,1963-05-28,0.0419,124.662,黃國隆,1
144483,0.796,1963,0.852,0.711,145720,0.111,0,5xFXTvnEe03SyvFpo6pEaE,0.000000,2,0.0695,-20.741,0,草螟弄雞公,23,1963-05-28,0.0697,124.273,"黃國隆, 王秋玉",2
150072,0.352,1992,0.381,0.353,316160,0.686,0,3KIuCzckjdeeVuswPo20mC,0.000000,11,0.0568,-9.103,1,DON'T BREAK MY HEART,35,1992-12-22,0.0395,200.341,黑豹,1


In [6]:
#get list of all genres in dataset
csv_path = "Resources/data_by_genres.csv"
genrelist_df = pd.read_csv(csv_path)
genrelist_df

,mode,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,21st century classical,0.979333,0.162883,1.602977e+05,0.071317,0.606834,0.361600,-31.514333,0.040567,75.336500,0.103783,27.833333,6
1,1,432hz,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.131000,-16.854000,0.076817,120.285667,0.221750,52.500000,5
2,1,8-bit,0.762000,0.712000,1.151770e+05,0.818000,0.876000,0.126000,-9.180000,0.047000,133.444000,0.975000,48.000000,7
3,1,[],0.651417,0.529093,2.328809e+05,0.419146,0.205309,0.218696,-12.288965,0.107872,112.857352,0.513604,20.859882,7
4,1,a cappella,0.676557,0.538961,1.906285e+05,0.316434,0.003003,0.172254,-12.479387,0.082851,112.110362,0.448249,45.820071,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2968,1,zolo,0.222625,0.547082,2.580991e+05,0.610240,0.143872,0.204206,-11.295878,0.061088,125.494919,0.596155,33.778943,9
2969,0,zouglou,0.161000,0.863000,2.063200e+05,0.909000,0.000000,0.108000,-5.985000,0.081300,119.038000,0.845000,58.000000,7
2970,1,zouk,0.263261,0.748889,3.060728e+05,0.622444,0.257227,0.089678,-10.289222,0.038778,101.965222,0.824111,46.666667,5
2971,0,zurich indie,0.993000,0.705667,1.984173e+05,0.172667,0.468633,0.179667,-11.453333,0.348667,91.278000,0.739000,0.000000,7


In [7]:
#create dataframe of artist and their genres
csv_path = "Resources/data_w_genres.csv"
genredata_df = pd.read_csv(csv_path)
genredata_df = genredata_df[["genres", "artists"]]
genredata_df["artists"] = genredata_df['artists'].str.strip("[]").str.replace("'","").str.replace('"',"")
genredata_df.sort_values('artists').head(40)


,genres,artists
9,"['emo rap', 'florida rap', 'sad rap', 'undergr...",$NOT
10,"['dark trap', 'meme rap']",$atori Zoom
11,[],$pyda
12,"['asian american hip hop', 'cali rap', 'west c...",$tupid Young
13,"['dark trap', 'new orleans rap', 'underground ...",$uicideBoy$
18,"['experimental hip hop', 'indie r&b', 'indie s...",((( O )))
19,[],(Con La Participación de Marc Anthony)
20,"['k-pop', 'k-pop girl group']",(G)I-DLE
21,"['alternative metal', 'industrial metal', 'nu ...",(Hed) P.E.
22,"['boy band', 'dance pop', 'europop', 'pop']",*NSYNC


In [8]:
# only run this code if you need to generate the csv

# #create a new column in newdata_df that lists the genres for a song based on the artist for that song. 
# combined_df = pd.merge(newdata_df, genredata_df, on = 'artists', how='left').sort_values(by='artists')
# combined_df.to_csv(r'Resources\data_artist_genre.csv')

csv_path = "Resources/data_artist_genre.csv"
combined_df = pd.read_csv(csv_path)
combined_df.sort_values('artists')

,Unnamed: 0,valence,year,acousticness,danceability,duration_ms,energy,explicit,id,instrumentalness,...,loudness,mode,name,popularity,release_date,speechiness,tempo,artists,collaborators,genres
0,38485,0.488,2020,0.810,0.847,120000,0.4480,1,0j8ppsOOawdPCJnSTcXgOy,0.000014,...,-10.140,1,GOSHA,78,2020-03-06,0.4160,79.975,$NOT,1,"['emo rap', 'florida rap', 'sad rap', 'undergr..."
1,155042,0.698,2018,0.874,0.716,119683,0.4140,1,4LS6xLmYwTUURADEIrmNaT,0.000042,...,-12.536,1,VINTAGE DIOR,59,2018-08-20,0.3400,147.974,$NOT,1,"['emo rap', 'florida rap', 'sad rap', 'undergr..."
2,140326,0.500,2018,0.814,0.819,120000,0.4620,1,3q8ms2oRMFsyxUYYKDiXIP,0.000016,...,-9.638,0,GOSHA,61,2018-09-14,0.4850,159.873,$NOT,1,"['emo rap', 'florida rap', 'sad rap', 'undergr..."
3,92341,0.351,2020,0.290,0.829,148093,0.6190,1,4X5AMbgW6whnAuDX10fhgq,0.000371,...,-7.258,1,Revenge,70,2020-09-24,0.1980,96.035,$NOT,1,"['emo rap', 'florida rap', 'sad rap', 'undergr..."
4,170607,0.631,2020,0.169,0.844,125853,0.7200,1,3PXi72ZtSqx1PZc40KS0Qj,0.000000,...,-7.418,0,Mean,65,2020-10-30,0.3000,99.917,"$NOT, Flo Milli",2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170647,159177,0.458,1963,0.987,0.241,193480,0.0437,0,4prhqrLXYMjHJ6vpRAlasx,0.000453,...,-24.571,1,藝旦調,23,1963-05-28,0.0443,85.936,黃國隆,1,[]
170648,144503,0.958,1963,0.720,0.816,147373,0.1870,0,7jSt9XJHRynGrrcnnIpOcf,0.000000,...,-20.530,0,丟丟銅仔,23,1963-05-28,0.0930,124.617,黃國隆,1,[]
170650,144483,0.796,1963,0.852,0.711,145720,0.1110,0,5xFXTvnEe03SyvFpo6pEaE,0.000000,...,-20.741,0,草螟弄雞公,23,1963-05-28,0.0697,124.273,"黃國隆, 王秋玉",2,NaN
170651,150072,0.352,1992,0.381,0.353,316160,0.6860,0,3KIuCzckjdeeVuswPo20mC,0.000000,...,-9.103,1,DON'T BREAK MY HEART,35,1992-12-22,0.0395,200.341,黑豹,1,"['chinese indie', 'chinese indie rock']"


In [9]:
# at this point, all songs with a single credited artist have been removed from the list.
# the remaining songs all have two or more credited performing artists.
# create new dataset of just the uncategorized songs, keeping only the columns necessary to rejoin to the main dataset later

nogenre_df = combined_df[combined_df['genres'].isnull()]
nogenre_df = nogenre_df.drop_duplicates(subset=['artists'])
nogenre_df = nogenre_df[["name", "artists"]]
nogenre_df["all artists"] = nogenre_df["artists"]
nogenre_df

,name,artists,all artists
4,Mean,"$NOT, Flo Milli","$NOT, Flo Milli"
6,Moon & Stars (feat. Maggie Lindemann),"$NOT, Maggie Lindemann","$NOT, Maggie Lindemann"
7,Human (feat. Night Lovell),"$NOT, Night Lovell","$NOT, Night Lovell"
8,Lovely,"$NOT, SUS Valentino","$NOT, SUS Valentino"
9,BERETTA (feat. Wifisfuneral),"$NOT, Wifisfuneral","$NOT, Wifisfuneral"
...,...,...,...
170429,ザ ダイアログ ウィズ ヴィヴラホーン,"猪俣猛, Ichiro Masuda","猪俣猛, Ichiro Masuda"
170502,I Thought It Was You,"笠井 紀美子, Herbie Hancock","笠井 紀美子, Herbie Hancock"
170580,相逢何必曾相識,"蔣志光, 韋綺珊, 王利名","蔣志光, 韋綺珊, 王利名"
170584,Touch You,"遠野高志, 加島優, 矢口恭介, ジミー, 百合絢斗, 田村唯, 明美圭一, 糸目幸士郎, 鹿谷樹","遠野高志, 加島優, 矢口恭介, ジミー, 百合絢斗, 田村唯, 明美圭一, 糸目幸士郎, 鹿谷樹"


In [10]:
#split 'artists' column along commas

splitartists_df = pd.concat([nogenre_df[["name", "all artists"]], nogenre_df['artists'].str.split(', ', expand=True)], axis=1)
splitartists_df

,name,all artists,0,1,2,3,4,5,6,7,...,30,31,32,33,34,35,36,37,38,39
4,Mean,"$NOT, Flo Milli",$NOT,Flo Milli,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,Moon & Stars (feat. Maggie Lindemann),"$NOT, Maggie Lindemann",$NOT,Maggie Lindemann,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,Human (feat. Night Lovell),"$NOT, Night Lovell",$NOT,Night Lovell,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,Lovely,"$NOT, SUS Valentino",$NOT,SUS Valentino,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,BERETTA (feat. Wifisfuneral),"$NOT, Wifisfuneral",$NOT,Wifisfuneral,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170429,ザ ダイアログ ウィズ ヴィヴラホーン,"猪俣猛, Ichiro Masuda",猪俣猛,Ichiro Masuda,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
170502,I Thought It Was You,"笠井 紀美子, Herbie Hancock",笠井 紀美子,Herbie Hancock,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
170580,相逢何必曾相識,"蔣志光, 韋綺珊, 王利名",蔣志光,韋綺珊,王利名,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
170584,Touch You,"遠野高志, 加島優, 矢口恭介, ジミー, 百合絢斗, 田村唯, 明美圭一, 糸目幸士郎, 鹿谷樹",遠野高志,加島優,矢口恭介,ジミー,百合絢斗,田村唯,明美圭一,糸目幸士郎,...,None,None,None,None,None,None,None,None,None,None


In [16]:
# create a function that will join genredata_df to splitartists_df once for each row, skipping the null values

def collabfunc():
    x = 2
    global splitartists_df
    if x <= 41:
        splitartists_df = splitartists_df.rename(columns={ splitartists_df.columns[x] : "artists"})
        splitartists_df = pd.merge(splitartists_df, genredata_df, on = "artists", how='left')
        splitartists_df = splitartists_df.rename(columns={"collaborator": "done" })
        x = x + 1
        print(x)
    
collabfunc()
splitartists_df

3


,name,all artists,artists,1,2,3,4,5,6,7,...,32,33,34,35,36,37,38,39,genres_x,genres_y
0,Mean,"$NOT, Flo Milli",$NOT,Flo Milli,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,"['emo rap', 'florida rap', 'sad rap', 'undergr...","['emo rap', 'florida rap', 'sad rap', 'undergr..."
1,Moon & Stars (feat. Maggie Lindemann),"$NOT, Maggie Lindemann",$NOT,Maggie Lindemann,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,"['emo rap', 'florida rap', 'sad rap', 'undergr...","['emo rap', 'florida rap', 'sad rap', 'undergr..."
2,Human (feat. Night Lovell),"$NOT, Night Lovell",$NOT,Night Lovell,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,"['emo rap', 'florida rap', 'sad rap', 'undergr...","['emo rap', 'florida rap', 'sad rap', 'undergr..."
3,Lovely,"$NOT, SUS Valentino",$NOT,SUS Valentino,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,"['emo rap', 'florida rap', 'sad rap', 'undergr...","['emo rap', 'florida rap', 'sad rap', 'undergr..."
4,BERETTA (feat. Wifisfuneral),"$NOT, Wifisfuneral",$NOT,Wifisfuneral,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,"['emo rap', 'florida rap', 'sad rap', 'undergr...","['emo rap', 'florida rap', 'sad rap', 'undergr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16365,ザ ダイアログ ウィズ ヴィヴラホーン,"猪俣猛, Ichiro Masuda",猪俣猛,Ichiro Masuda,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,['classic japanese jazz'],['classic japanese jazz']
16366,I Thought It Was You,"笠井 紀美子, Herbie Hancock",笠井 紀美子,Herbie Hancock,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,"['classic japanese jazz', 'japanese vocal jazz']","['classic japanese jazz', 'japanese vocal jazz']"
16367,相逢何必曾相識,"蔣志光, 韋綺珊, 王利名",蔣志光,韋綺珊,王利名,None,None,None,None,None,...,None,None,None,None,None,None,None,None,['classic cantopop'],['classic cantopop']
16368,Touch You,"遠野高志, 加島優, 矢口恭介, ジミー, 百合絢斗, 田村唯, 明美圭一, 糸目幸士郎, 鹿谷樹",遠野高志,加島優,矢口恭介,ジミー,百合絢斗,田村唯,明美圭一,糸目幸士郎,...,None,None,None,None,None,None,None,None,[],[]
